In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Pisco,-13.7000,-76.2167,75.00,83,75,9.22,PE,2/6/2021 17:11
1,1,Pevek,69.7008,170.3133,-1.64,86,100,2.24,RU,2/6/2021 17:10
2,2,Ushuaia,-54.8000,-68.3000,53.60,50,20,21.85,AR,2/6/2021 17:11
3,3,Turukhansk,65.8167,87.9833,-5.75,91,100,17.05,RU,2/6/2021 17:09
4,4,Coruripe,-10.1256,-36.1756,86.92,58,92,18.23,BR,2/6/2021 17:11


In [3]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Pisco,-13.7000,-76.2167,75.00,83,75,9.22,PE,2/6/2021 17:11
4,4,Coruripe,-10.1256,-36.1756,86.92,58,92,18.23,BR,2/6/2021 17:11
6,6,Cam Ranh,11.9214,109.1591,75.20,73,40,10.36,VN,2/6/2021 17:11
11,11,Port Blair,11.6667,92.7500,78.98,76,0,7.99,IN,2/6/2021 17:11
13,13,Tual,-5.6667,132.7500,83.97,75,100,21.00,ID,2/6/2021 17:09
14,14,Rikitea,-23.1203,-134.9692,77.47,75,100,13.15,PF,2/6/2021 17:11
16,16,Kikwit,-5.0410,18.8162,75.27,89,92,1.57,CD,2/6/2021 17:11
20,20,Wajid,3.8093,43.2471,84.99,24,0,13.15,SO,2/6/2021 17:11
23,23,Avarua,-21.2078,-159.7750,75.20,94,20,2.30,CK,2/6/2021 17:11
25,25,Sola,-13.8833,167.5500,81.70,75,5,6.26,VU,2/6/2021 17:09


In [8]:
preferred_cities_df.count()

City_ID       202
City          202
Lat           202
Lng           202
Max Temp      202
Humidity      202
Cloudiness    202
Wind Speed    202
Country       199
Date          202
dtype: int64

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Pisco,PE,75.00,-13.7000,-76.2167,
4,Coruripe,BR,86.92,-10.1256,-36.1756,
6,Cam Ranh,VN,75.20,11.9214,109.1591,
11,Port Blair,IN,78.98,11.6667,92.7500,
13,Tual,ID,83.97,-5.6667,132.7500,
14,Rikitea,PF,77.47,-23.1203,-134.9692,
16,Kikwit,CD,75.27,-5.0410,18.8162,
20,Wajid,SO,84.99,3.8093,43.2471,
23,Avarua,CK,75.20,-21.2078,-159.7750,
25,Sola,VU,81.70,-13.8833,167.5500,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [43]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [44]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Pisco,PE,75.00,-13.7000,-76.2167,Hotel San Isidro Oficial
4,Coruripe,BR,86.92,-10.1256,-36.1756,Pousada e Hotel Litoral Sul
6,Cam Ranh,VN,75.20,11.9214,109.1591,OYO 598 Peony Hotel
11,Port Blair,IN,78.98,11.6667,92.7500,Welcomhotel Bay Island Port Blair
13,Tual,ID,83.97,-5.6667,132.7500,Grand Vilia Hotel
14,Rikitea,PF,77.47,-23.1203,-134.9692,Pension Maro'i
16,Kikwit,CD,75.27,-5.0410,18.8162,Guest L'Amour
20,Wajid,SO,84.99,3.8093,43.2471,xamdihotelresturant114
23,Avarua,CK,75.20,-21.2078,-159.7750,Paradise Inn
25,Sola,VU,81.70,-13.8833,167.5500,Leumerus Bungalows


In [36]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [38]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [39]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [40]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))